In [1]:
import os
import numpy as np
from InputDataProcessor import InputDataProcessor 
from GenericModel import GenericModel
from distributeFunctions import distribute_602020
from dataFilters import csv_weather_data_prediction_filter, csv_weather_data_classification_filter
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from ModelConfig import ModelConfig

2024-05-13 03:54:27.566453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 03:54:28.950640: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-13 03:54:28.950765: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
idp=InputDataProcessor(bucket_name='aiml_harveen', use_case='weather_elbasan/classification', file_name='elbasan2021.csv')

In [3]:
idp.prepareCsvModelInputData(csv_weather_data_classification_filter,distribute_602020)

In [4]:
os.environ["AIP_TRAINING_DATA_URI"] = "gs://aiml_harveen/weather_elbasan/classification/training_data/elbasan2021.csv"
os.environ["AIP_VALIDATION_DATA_URI"] = "gs://aiml_harveen/weather_elbasan/classification/dev_data/elbasan2021.csv"
os.environ["AIP_TEST_DATA_URI"] = "gs://aiml_harveen/weather_elbasan/classification/test_data/elbasan2021.csv"
os.environ["AIP_MODEL_DIR"] = "gs://aiml_harveen/weather_elbasan/classification/weather_model"

In [5]:
modelConfigList=[
    ModelConfig(
    model_type="classification",
    layers=["relu","relu","sigmoid"],
    regularizers=[0.02,0.02,0.02],
    neurons=[32,16,1],
    epochs=20,
    loss=BinaryCrossentropy(),
    optimizer=Adam(learning_rate=0.003)),
    
#    ModelConfig(
#    model_type="classification",
#    layers=["relu","relu","relu","relu","sigmoid"],
#    regularizers=[0.02,0.02,0.02,0.02,0.02],
#    neurons=[32,32,16,16,1],
#    epochs=20,
#    loss=BinaryCrossentropy(),
#    optimizer=Adam(learning_rate=0.003))
]

2024-05-13 03:54:33.033806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-13 03:54:33.033846: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-13 03:54:33.033873: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20240502-111332): /proc/driver/nvidia/version does not exist
2024-05-13 03:54:33.035111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow 

In [6]:
trainErrAry=np.zeros([len(modelConfigList)])
validationErrAry=np.zeros([len(modelConfigList)])
testErrAry=np.zeros([len(modelConfigList)])

In [7]:
for idx,modelConfig in enumerate(modelConfigList):
    gm=GenericModel(modelConfig)
    gm.train()
    trainErrAry[idx] = gm.train_err
    gm.validate()
    validationErrAry[idx] = gm.validation_err
    gm.test()
    testErrAry[idx] = gm.test_err


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                224       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________
3/3 [==============================] - 0s 2ms/step
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 

In [8]:
trainErrAry

array([0.16055046])

In [9]:
validationErrAry

array([0.08219178])

In [10]:
testErrAry

array([0.1369863])